In [1]:
from tool import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 전년 대비 PC 공급 수량 시각화

In [2]:
with open('D:/Data/GSCP raw data.bin', 'rb') as f:
    sp = pickle.load(f)

lt_wk = sp['Ref'].max()
dt_obj = datetime.datetime.strptime(lt_wk[:10], '%Y-%m-%d')
thismonth = str(dt_obj.isocalendar().year) +  '-' + '{0:02d}'.format(get_month_from_date(dt_obj))
thismonth

'2023-07'

In [9]:
def plot_odm_shipment_for_recent_three_years(vendor, wk_name, thismonth, ver):
    sp = monthly_sum(get_sp_from_GSCP_DB(wk_name, ver, vendor), 'Series').loc[:, thismonth:]
    sp = sp.stack().reset_index()
    sp.rename(columns={'level_1':'Year', 0:'QTY'}, inplace=True)
    sp.loc[:, ['Year', 'Month']] = sp['Year'].str.split('-', expand=True).values
    sp['Year'] = sp['Year'].astype(int)
    sp['Month'] = sp['Month'].astype(int)
    sp = sp[sp['Year']==int(thismonth.split('-')[0])]
    sp = sp.sort_values(['Year', 'Month'])
    sp['Month'] = sp['Month'].apply(lambda x:datetime.datetime.strptime('{0:02d}'.format(x), '%m').strftime('%b'))

    with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
        sr = pickle.load(f)

    cond = ((sr['Ship Year'] == int(thismonth.split('-')[0])) & (sr['Ship Month'] == int(thismonth.split('-')[-1]))) # 이번달 실적
    sr = sr[~cond] # 이번달 실적은 제외함
    cond = sr['Ship Year'] >= (int(thismonth.split('-')[0]) - 2) # 재작년부터 현재까지의 실적
    sr = sr[cond]
    sr = sr.pivot_table('Ship', index=['Series', 'Ship Year', 'Ship Month'], aggfunc='sum').fillna(0).reset_index()
    sr = sr.rename({'Ship Year':'Year', 'Ship Month':'Month', 'Ship':'QTY'}, axis=1)
    sr = sr.sort_values(['Year', 'Month'])
    sr['Month'] = sr['Month'].apply(lambda x:datetime.datetime.strptime('{0:02d}'.format(int(x)), '%m').strftime('%b'))
    result = pd.concat([sr, sp]).reset_index(drop=True)
    fig = px.histogram(result, x='Month', y='QTY', color='Year', text_auto=True, barmode='group')
    return fig.update_xaxes(categoryarray= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [12]:
plot_odm_shipment_for_recent_three_years('Pegatron', lt_wk, thismonth, 'Final')

In [150]:
sp = monthly_sum(get_sp_from_GSCP_DB(get_weekname(datetime.date.today()), ver, 'Wingtech'), 'Series').loc[:, thismonth:]
sp = sp.stack().reset_index()
sp.rename(columns={'level_1':'Year', 0:'QTY'}, inplace=True)
sp.loc[:, ['Year', 'Month']] = sp['Year'].str.split('-', expand=True).values
sp = sp[sp['Year']==thismonth.split('-')[0]]
sp['Month'] = sp['Month'].apply(lambda x:datetime.datetime.strptime(x, '%m').strftime('%b'))
sp

,Series,Year,QTY,Month
0,10A30Q,2023,0.0,Jun
1,10A30Q,2023,6260.0,Jul
2,10A30Q,2023,600.0,Aug
3,10A30Q,2023,300.0,Sep
4,10A30Q,2023,10600.0,Oct
5,10A30Q,2023,10000.0,Nov
6,10A30Q,2023,10000.0,Dec


In [154]:
with open(f'D:/Data/Wingtech shipment result DB.bin', 'rb') as f:
    sr = pickle.load(f)

cond = ((sr['Ship Week year'] == thismonth.split('-')[0]) & (sr['Ship Week month'] == thismonth.split('-')[-1])) # 이번달 실적
sr = sr[~cond] # 이번달 실적은 제외함
cond = sr['Ship Week year'] >= str(int(thismonth.split('-')[0]) - 2) # 재작년부터 현재까지의 실적
sr = sr[cond]
sr = sr.pivot_table('Ship', index=['Series', 'Ship Week year', 'Ship Week month'], aggfunc='sum').fillna(0).reset_index()
sr = sr.rename({'Ship Week year':'Year', 'Ship Week month':'Month', 'Ship':'QTY'}, axis=1)
sr['Month'] = sr['Month'].apply(lambda x:datetime.datetime.strptime(x, '%m').strftime('%b'))
sr

,Series,Year,Month,QTY
0,10A30Q,2022,Jul,1760.0
1,10A30Q,2022,Aug,1080.0
2,10A30Q,2022,Nov,1320.0
3,10A30Q,2022,Dec,680.0
4,10A30Q,2023,Jan,2020.0
5,10A30Q,2023,Feb,560.0
6,10A30Q,2023,Mar,220.0


In [161]:
result = pd.concat([sr, sp], axis=0).reset_index(drop=True)
result

,Series,Year,Month,QTY
0,10A30Q,2022,Jul,1760.0
1,10A30Q,2022,Aug,1080.0
2,10A30Q,2022,Nov,1320.0
3,10A30Q,2022,Dec,680.0
4,10A30Q,2023,Jan,2020.0
5,10A30Q,2023,Feb,560.0
6,10A30Q,2023,Mar,220.0
7,10A30Q,2023,Jun,0.0
8,10A30Q,2023,Jul,6260.0
9,10A30Q,2023,Aug,600.0


In [162]:
fig = px.histogram(result, x='Month', y='QTY', color='Year', text_auto=True, barmode='group')
fig

# 업체명, GSCP Ver, Plan Week  별로 SP 가져오는 함수

In [2]:
def sp_get(ver, vendor, week_name):
    with open('D:/Data/GSCP raw data.bin', 'rb') as f:
        sp_all = pickle.load(f)
    # GSCP ver과 기준시점 주차와 vendor명으로 1차 추출
    c1 = (sp_all['Ver'] == ver)
    c2 = (sp_all['Ref'] == week_name)
    c3 = (sp_all['From Site'].isin(vendor))
    sp = sp_all[c1 & c2 & c3].reset_index(drop=True).copy()
    if sp.shape[0] != 0:
        sp = sp[sp['Updated_at'].isin(sp[['Ref', 'Ver', 'Updated_at']].drop_duplicates().groupby(['Ref', 'Ver'])['Updated_at'].max().unstack()[ver])].reset_index(drop=True)
        sp = sp[['Ref', 'Mapping Model.Suffix', 'To Site'] + get_weeklist(sp)].fillna(0)
        sp = sp.groupby(['Ref', 'Mapping Model.Suffix', 'To Site']).sum(numeric_only=True)
        min_wk = sp.sum() > 0
        min_wk = min_wk[min_wk == True].index.min()
        sp = sp.loc[:, min_wk:]
    return sp

In [13]:
# 함수에 사용할 Argument
ver = 'Final'
vendor = ['Quanta']
planweeks = get_weekname(datetime.date.today())
planweeks = [get_weekname_from(planweeks, i) for i in range(-12, 1)]
col1 = ['Ref', 'Mapping Model.Suffix', 'To Site']

## planweeks 를 돌며 주차별 sp 를 누적함

In [14]:
for i, wk_name in enumerate(tqdm(planweeks)):
    if i == 0: # 누적할 sp의 초기값 셋팅
        acc_sp = sp_get(ver, vendor, wk_name)
    else:
        sp = sp_get(ver, vendor, wk_name)
        if sp.shape[0] == 0:
            continue
        deleted_wk = []
        for item in acc_sp.columns:
            if item not in sp.columns:
                deleted_wk.append(item)
                
        org_idx = acc_sp.reset_index().set_index(['Mapping Model.Suffix', 'To Site']).index
        new_idx = sp.reset_index().set_index(['Mapping Model.Suffix', 'To Site']).index
        intersecting_idx = org_idx.intersection(new_idx)
        diff_idx_org = org_idx.difference(intersecting_idx) # 첫 번째 인덱스에서 2번째 인덱스를 뺀 차집합 인덱스
        
        # 누적 데이터프레임에서 동일 index값이 가장 최근에 언제 주차에 있었는지 참조하기 위한 Series
        find_most_recent_week = acc_sp.reset_index()[col1].groupby(['Mapping Model.Suffix', 'To Site'])['Ref'].max()
        # 누적 데이터프레임에서 신규SP에 없는 row 중에서 가장 최근 주차의 행을 지정하기 위한 멀티인덱스를 만들기 위한 데이터프레임1
        idx_frame1 = find_most_recent_week[diff_idx_org].reset_index()[col1]
        m_idx1 = pd.MultiIndex.from_frame(idx_frame1)
        add_df = acc_sp.loc[m_idx1, :get_weekname_from(wk_name, -2)] # 이전주차까지의 값만 슬라이싱하여 추가할 df를 만듬
        for w in find_most_recent_week[diff_idx_org]:  # 누적 데이터프레임에서 차집합 인덱스를 반복하면서
            if w < get_weekname_from(wk_name, -1):     # 해당 최근주차가 지난주보다 작을 때
                add_df.loc[w, w:] = 0                  # 해당 최근주차부터 이후 주차의 값은 '0' 으로 만듬(왜냐하면, 기존값은 실적이 아니므로)
        add_df = add_df[add_df.sum(axis=1) > 0]
        
        # deleted_wk 에 값이 있을 경우 df에 이전주차까지의 누적 데이터프레임의 삭제된 주차들의 선적 실적을 추가하는 작업
        if len(deleted_wk) > 0:
            # 누적 데이터프레임과 신규SP와 공통 row중에서 가장 최근 주차의 행을 지정하기 위한 멀티인덱스를 만들기 위한 데이터프레임1
            idx_frame2 = find_most_recent_week[intersecting_idx].reset_index()[col1]
            idx_frame3 = idx_frame2.copy()
            idx_frame3['Ref'] = wk_name
            m_idx2 = pd.MultiIndex.from_frame(idx_frame2)
            m_idx3 = pd.MultiIndex.from_frame(idx_frame3)
            sp.loc[m_idx3, deleted_wk] = acc_sp.loc[m_idx2, deleted_wk].values
            
        sp = pd.concat([add_df, sp]).reset_index()
        sp['Ref'] = wk_name
        sp = sp.set_index(col1)
        acc_sp = pd.concat([acc_sp, sp])

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  4.09it/s]


In [15]:
# 시각화하기 좋은 데이터프레임으로 변형
df = acc_sp.reset_index().fillna(0)
df.insert(1, 'Series', df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model))
df.insert(3, 'Region', df['To Site'].replace(site_map).replace(country_map))
df = monthly_sum(df, ['Ref', 'Series', 'Region'])
df.columns.name = 'Month'
df = df.loc[:, df.sum() > 0].stack().reset_index()
df.rename({0:'QTY'}, axis=1, inplace=True)

## 1. 최근 4주간 SP 현황

In [16]:
fig = go.Figure()
clr = px.colors.sequential.Reds
for wk in df['Ref'].unique()[-4:]:
    df_wk = df[df['Ref'] == wk]
    fig.add_trace(go.Histogram(histfunc='sum', x=df_wk['Month'], y=df_wk['QTY'], name=wk, )) # 데이터프레임의 각 row의 값을 누적하여 Bar graph를 나타날 때 사용
fig.update_layout(colorway=clr, width=1000, barmode='group')
fig.update_xaxes(nticks=len(df_wk['Month'].unique()) + 1, tickangle=-45)
fig # 최근 4주의 월별 SP 현황

## 2. 주차별 SP 변동 현황

In [17]:
# 주차별 합계 수량
px.histogram(df, x='Ref', y='QTY', barmode='relative', text_auto='QTY')

In [18]:
# 월별 분포
px.histogram(df, x='Ref', y='QTY', color='Month', barmode='relative')

In [19]:
# Series별 분포
px.histogram(df, x='Ref', y='QTY', color='Series', barmode='relative')

In [20]:
# Region별 분포
px.histogram(df, x='Ref', y='QTY', color='Region', barmode='relative')

## 3. 각 시리즈 모델별 SP 변동 현황

In [21]:
fig = make_subplots(rows=(len(df['Series'].unique()) + 3) // 4, cols=4, subplot_titles=df['Series'].unique(), shared_xaxes=True, horizontal_spacing=0.03, vertical_spacing=0.05)
i = 0
row_num = 1
col_num = 1
for sr in df['Series'].unique():
    df_sr = df[df['Series'] == sr]
    for mon in df_sr['Month'].unique():
        df1 = df_sr[df_sr['Month'] == mon]
        fig.add_trace(go.Histogram(histfunc='sum', x=df1['Ref'], y=df1['QTY'], name=mon, hovertext=df1['Month']), row=row_num, col=col_num)
    i += 1
    row_num = i // 4 + 1
    col_num = i % 4 + 1
fig.update_layout(barmode='relative', height=1000, showlegend=False)
fig.update_xaxes(tickangle=-65)

In [100]:
fig = make_subplots(rows=(len(df['Series'].unique()) + 3) // 4, cols=4, subplot_titles=df['Series'].unique(), shared_xaxes=True, horizontal_spacing=0.03, vertical_spacing=0.05)
i = 0
row_num = 1
col_num = 1
for sr in df['Series'].unique():
    df_sr = df[df['Series'] == sr]
    for reg in df_sr['Region'].unique():
        df1 = df_sr[df_sr['Region'] == reg]
        fig.add_trace(go.Histogram(histfunc='sum', x=df1['Ref'], y=df1['QTY'], name=mon, hovertext=df1['Region']), row=row_num, col=col_num)
    i += 1
    row_num = i // 4 + 1
    col_num = i % 4 + 1
fig.update_layout(barmode='relative', height=1000, showlegend=False)
fig.update_xaxes(tickangle=-65)

In [78]:
fig = make_subplots(rows=(len(df['Region'].unique()) + 2) // 3, cols=3, subplot_titles=df['Region'].unique(), shared_xaxes=True, horizontal_spacing=0.02, vertical_spacing=0.03)
i = 0
row_num = 1
col_num = 1
for rg in df['Region'].unique():
    df_rg = df[df['Region'] == rg]
    for mon in df_rg['Month'].unique():
        df1 = df_rg[df_rg['Month'] == mon]
        fig.add_trace(go.Histogram(histfunc='sum', x=df1['Ref'], y=df1['QTY'], name=mon, hovertext=df1['Month']), row=row_num, col=col_num)
    i += 1
    row_num = i // 3 + 1
    col_num = i % 3 + 1
fig.update_layout(barmode='relative', height=1000, showlegend=False)
fig.update_xaxes(tickangle=-45)

## 4. 특정 모델, 지역의 SP 변동 현황

In [79]:
# 검색 조건 설정
series_list = ['15U40Q']
region = ['KOREA']

In [80]:
fig = px.histogram(df[df['Region'].isin(region) & df['Series'].isin(series_list)], x='Ref', y='QTY', color='Month', width=600)
fig.update_xaxes(tickangle=-65, )
fig.update_layout(title=f'{region[0]}, {series_list} {ver} SP History')